# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 82 new papers today
          36 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/35 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.13311


extracting tarball to tmp_2211.13311...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.13318


extracting tarball to tmp_2211.13318...

...

extracting tarball to tmp_2211.14225...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.14262


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.14225/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tables/table_comp_squicc_miret.tex' from 'tmp_2211.14225/tables/table_comp_squicc_miret.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tables/table_comp_kerr.tex' from 'tmp_2211.14225/tables/table_comp_kerr.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/

extracting tarball to tmp_2211.14262...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.13318-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.13318) | **High Resolution Study of Planetesimal Formation by Gravitational  Collapse of Pebble Clouds**  |
|| Brooke Polak, <mark>Hubert Klahr</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| Planetary embryos are built through the collisional growth of 10-100 km sized objects called planetesimals, a formerly large population of objects, of which asteroids, comets and Kuiper-Belt objects represent the leftovers from planet formation in our solar system. Here, we follow the paradigm that turbulence created over-dense pebble clouds, which then collapse under their own self-gravity. We use the multi-physics code GIZMO to model the pebble cloud density as a continuum, with a polytropic equation of state to account for collisional interactions and capturing the phase transition to a quasi-incompressible solid object, i.e. a planetesimal in hydrostatic equilibrium. Thus we study cloud collapse effectively at the resolution of the forming planetesimals, allowing us to derive an initial mass function for planetesimals in relation to the total pebble mass of the collapsing cloud. The redistribution of angular momentum in the collapsing pebble cloud is the main mechanism leading to multiple fragmentation. The angular momentum of the pebble cloud and thus the centrifugal radius increases with distance to the sun, but the solid size of the forming planetesimals is constant. Therefore we find that with increasing distance to the sun, the number of forming planetesimals per pebble cloud increases. For all distances the formation of binaries occurs within higher hierarchical systems. The size distribution is top heavy and can be described with a Gaussian distribution of planetesimal mass. For the asteroid belt, we can infer a most likely size of 125 km, all stemming from pebble clouds of equivalent size 152 km. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.13426-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.13426) | **PHANGS-JWST First Results: Dust embedded star clusters in NGC 7496  selected via 3.3 $μ$m PAH emission**  |
|| Jimena Rodriguez, et al. -- incl., <mark>Janice Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Daizhong Liu</mark>, <mark>Kathryn Kreckel</mark>, <mark>Annie Hughes</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *12 pages, 6 figures*|
|**Abstract**| The earliest stages of star formation occur enshrouded in dust and are not observable in the optical. Here we leverage the extraordinary new high-resolution infrared imaging from JWST to begin the study of dust-embedded star clusters in nearby galaxies throughout the local volume. We present a technique for identifying dust-embedded clusters in NGC 7496 (18.7 Mpc), the first galaxy to be observed by the PHANGS-JWST Cycle 1 Treasury Survey. We select sources that have strong 3.3$\mu$m PAH emission based on a $\rm F300M-F335M$ color excess, and identify 67 candidate embedded clusters. Only eight of these are found in the PHANGS-HST optically-selected cluster catalog and all are young (six have SED-fit ages of $\sim1$ Myr). We find that this sample of embedded cluster candidates may significantly increase the census of young clusters in NGC 7496 from the PHANGS-HST catalog -- the number of clusters younger than $\sim$2 Myr could be increased by a factor of two. Candidates are preferentially located in dust lanes, and are coincident with peaks in PHANGS-ALMA CO (2-1) maps. We take a first look at concentration indices, luminosity functions, SEDs spanning from 2700A to 21$\mu$m, and stellar masses (estimated to be between $\sim10^4-10^5 M_{\odot}$). The methods tested here provide a basis for future work to derive accurate constraints on the physical properties of embedded clusters, characterize the completeness of cluster samples, and expand analysis to all 19 galaxies in the PHANGS-JWST sample, which will enable basic unsolved problems in star formation and cluster evolution to be addressed. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.13811-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.13811) | **A global view on star formation: The GLOSTAR Galactic plane survey. VII.  Supernova remnants in the Galactic longitude range $28^\circ<l<36^\circ$**  |
|| R. Dokara, et al. -- incl., <mark>H. Beuther</mark>, <mark>P. Müller</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *To be published in A&A. 21 pages, 15 figures*|
|**Abstract**| Context. While over 1000 supernova remnants (SNRs) are estimated to exist in the Milky Way, only less than 400 have been found to date. In the context of this apparent deficiency, more than 150 SNR candidates were recently identified in the D-configuration Very Large Array (VLA-D) continuum images of the 4--8 GHz global view on star formation (GLOSTAR) survey, in the Galactic longitude range $-2^\circ<l<60^\circ$. Aims. We attempt to find evidence of nonthermal synchrotron emission from 35 SNR candidates in the region of Galactic longitude range $28^\circ<l<36^\circ$, and also to study the radio continuum emission from the previously confirmed SNRs in this region. Methods. Using the short-spacing corrected GLOSTAR VLA-D+Effelsberg images, we measure ${\sim}6$ GHz total and linearly polarized flux densities of the SNR candidates and the SNRs that were previously confirmed. We also attempt to determine the spectral indices by measuring flux densities from complementary Galactic plane surveys and from the temperature-temperature plots of the GLOSTAR-Effelsberg images. Results. We provide evidence of nonthermal emission from four candidates that have spectral indices and polarization consistent with a SNR origin, and, considering their morphology, we are confident that three of these (G28.36+0.21, G28.78-0.44, and G29.38+0.10) are indeed SNRs. However, about $25\%$ of the candidates have spectral index measurements that indicate thermal emission, and the rest of them are too faint to have a good constraint on the spectral index yet. Conclusions. Additional observations at longer wavelengths and higher sensitivities will shed more light on the nature of these candidates. A simple Monte-Carlo simulation reiterates the view that future studies must persist with the current strategy of searching for SNRs with small angular size to solve the problem of the Milky Way's missing SNRs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14132-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14132) | **The Gaia-ESO Survey: probing the lithium abundances in old metal-rich  dwarf stars in the Solar vicinity**  |
|| M. L. L. Dantas, et al. -- incl., <mark>G. Guiglion</mark>, <mark>U. Heiter</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *8 pages, 3 figures. Accepted for publication in A&A Letters. Abridged abstract to fit ArXiv's requirements*|
|**Abstract**| We test a scenario in which radial migration could affect the Li abundance pattern of dwarf stars in the solar neighbourhood. This may confirm that the Li abundance in these stars can not serve as a probe for the Li abundance in the interstellar medium. We use the high-quality data (including Li abundances) from the 6th internal Data Release of the Gaia-ESO survey. In this sample, we group stars by similarity in chemical abundances via hierarchical clustering. Our analysis treats both measured Li abundances and upper limits. The Li envelope of the previously identified radially migrated stars is well below the benchmark meteoritic value (<3.26 dex); the star with the highest detected abundance has A(Li) = 2.76 dex. This confirms the previous trends observed for old dwarf stars (median ages $\sim$ 8 Gyr), where Li decreases for [Fe/H]$\gtrsim$0. This result acts as supporting evidence that the abundance of Li measured in the upper envelope of old dwarf stars should not be considered a proxy for the interstellar medium Li. Our scenario also indicates that the stellar yields for [M/H]>0 should not be decreased, as recently proposed in the literature. Our study backs the recent studies that claimed that old dwarfs on the hot side of the dip are efficient probes of the ISM abundance of Li, provided atomic diffusion does not lower significantly the initial Li abundance in the atmospheres of metal-rich objects. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14131-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14131) | **The uncertain interstellar medium of high-redshift quiescent galaxies:  Impact of methodology**  |
|| Raphaël Gobat, et al. -- incl., <mark>Daizhong Liu</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *7 pages, 5 figures, 2 tables; accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| How much gas and dust is contained in high-redshift quiescent galaxies (QGs) is currently an open question with relatively few and contradictory answers, as well as important implications for our understanding of the nature of star formation quenching processes at cosmic noon. Here we revisit far-infrared (FIR) observations of the REQUIEM-ALMA sample of six z = 1.6 - 3.2 QGs strongly lensed by intermediate-redshift galaxy clusters. We measured their continuum emission using priors obtained from high resolution near-infrared (NIR) imaging, as opposed to focusing on point-source extraction, converted it into dust masses using a FIR dust emission model derived from statistical samples of QGs, and compared the results to those of the reference work. We find that, while at least the most massive sample galaxy is indeed dust-poor, the picture is much more nuanced than previously reported. In particular, these more conservative constraints remain consistent with high dust fractions in early QGs. We find that these measurements are very sensitive to the adopted extraction method and conversion factors: the use of an extended light model to fit the FIR emission increases the flux of detections by up to 50% and the upper limit by up to a factor 6. Adding the FIR-to-dust conversion, this amounts to an order of magnitude difference in dust fraction, casting doubts on the power of these data to discriminate between star formation quenching scenarios. Unless these are identified by other means, mapping the dust and gas in high-redshift QGs will continue to require somewhat costly observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14200-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14200) | **Constraints on the Model of Gamma-ray Bursts and Implications from GRB  221009A: GeV gamma rays v.s. High-energy Neutrinos**  |
|| <mark>Ruo-Yu Liu</mark>, <mark>Hai-Ming Zhang</mark>, Xiang-Yu Wang |
|*Appeared on*| *2022-11-28*|
|*Comments*| *10 pages, 6 figures; comments are welcome*|
|**Abstract**| Gamma-ray bursts (GRB) are generally believed to be efficient particle accelerators. In the presence of energetic protons in a GRB jet, interactions between these protons and intense radiation field of the GRB are supposed to induce electromagnetic cascade. Electrons/positrons generated in the cascade will produce an additional spectrum of robust feature, which is in the form of a power-law distribution up to GeV regime with an index of $\lesssim 2$. We suggest that measurements of Fermi-LAT at GeV band can provide independent constraints on the key GRB model parameters such as the dissipation radius, the jet's bulk Lorentz factor, and the baryon loading factor. Taking GRB 221009A, the brightest GRB ever detected, as an example, we show that the constraints from GeV gamma-ray emission may be more stringent than that from the neutrino observation, providing us a deep insight into the origin of GRBs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14262-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14262) | **Hybrid $α$-attractors, primordial black holes and gravitational  wave backgrounds**  |
|| Matteo Braglia, <mark>Andrei Linde</mark>, Renata Kallosh, Fabio Finelli |
|*Appeared on*| *2022-11-28*|
|*Comments*| *39 pages, 12 figures*|
|**Abstract**| We investigate the two-stage inflation regime in the theory of hybrid cosmological $\alpha$-attractors. The spectrum of inflationary perturbations is compatible with the latest Planck/BICEP/Keck results, thanks to the attractor properties of the model. However, at smaller scales, it may have a very high peak of controllable width and position, leading to a copious production of primordial black holes (PBH) and generation of a stochastic background of gravitational waves (SGWB). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.14225-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.14225) | **Significance Mode Analysis (SigMA) for hierarchical structures. An  application to the Sco-Cen OB association**  |
|| Sebastian Ratzenböck, et al. -- incl., <mark>Josefa E. Großschedl</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *Submitted after first revision to A&A*|
|**Abstract**| We present a new clustering method, Significance Mode Analysis (SigMA), to extract co-spatial and co-moving stellar populations from large-scale surveys such as ESA Gaia. The method studies the topological properties of the density field in the multidimensional phase space. We validate SigMA on simulated clusters and find that it outperforms competing methods, especially in cases where many clusters are closely spaced. We apply the new method to Gaia DR3 data of the closest OB association to Earth, Scorpio-Centaurus (Sco-Cen), and find more than 13,000 co-moving young objects, with about 19% of these having a sub-stellar mass. SigMA finds 37 co-moving clusters in Sco-Cen. These clusters are independently validated by their narrow HRD sequences and, to a certain extent, by their association with massive stars too bright for Gaia, hence unknown to SigMA. We compare our results with similar recent work and find that the SigMA algorithm recovers richer populations, is able to distinguish clusters with velocity differences down to about 0.5 km s$^{-1}$, and reaches cluster volume densities as low as 0.01 sources/pc$^3$. The 3D distribution of these 37 coeval clusters implies a larger extent and volume for the Sco-Cen OB association than typically assumed in the literature. Additionally, we find the association to be more actively star-forming and dynamically more complex than previously thought. We confirm that the star-forming molecular clouds in the Sco-Cen region, namely, Ophiuchus, L134/L183, Pipe Nebula, Corona Australis, Lupus, and Chamaeleon, are part of the Sco-Cen The application of SigMA to Sco-Cen demonstrates that advanced machine learning tools applied to the superb Gaia data allows to construct an accurate census of the young populations, to quantify their dynamics, and to reconstruct the recent star formation history of the local Milky Way. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.13997-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.13997) | **PHANGS-JWST First Results: A combined HST and JWST analysis of the  nuclear star cluster in NGC 628**  |
|| Nils Hoyer, et al. -- incl., <mark>Francesca Pinna</mark>, <mark>Albrecht W. H. Kamlah</mark>, <mark>Francisco Nogueras-Lara</mark>, <mark>Anja Feldmeier-Krause</mark>, <mark>Nadine Neumayer</mark>, <mark>Eva Schinnerer</mark>, <mark>Justus Neumann</mark>, <mark>Frank Bigiel</mark>, <mark>Daizhong Liu</mark> |
|*Appeared on*| *2022-11-28*|
|*Comments*| *26 pages, 10 figures, 6 tables. Accepted for publication by ApJL*|
|**Abstract**| We combine archival HST and new JWST imaging data, covering the ultraviolet to mid-infrared regime, to morphologically analyze the nuclear star cluster (NSC) of NGC 628, a grand-design spiral galaxy. The cluster is located in a 200 pc x 400 pc cavity, lacking both dust and gas. We find roughly constant values for the effective radius (r_eff ~ 5 pc) and ellipticity ({\epsilon} ~ 0.05), while the S\'ersic index (n) and position angle (PA) drop from n ~ 3 to ~ 2 and PA ~ 130{\deg} to 90{\deg}, respectively. In the mid-infrared, r_eff ~ 12pc, {\epsilon} ~ 0.4, and n ~ 1-1.5, with the same PA ~ 90{\deg}. The NSC has a stellar mass of log10 (M_nsc / M_Sun) = 7.06 +- 0.31, as derived through B-V, confirmed when using multi-wavelength data, and in agreement with the literature value. Fitting the spectral energy distribution, excluding the mid-infrared data, yields a main stellar population's age of (8 +- 3) Gyr with a metallicity of Z = 0.012 +- 0.006. There is no indication of any significant star formation over the last few Gyr. Whether gas and dust were dynamically kept out or evacuated from the central cavity remains unclear. The best-fit suggests an excess of flux in the mid-infrared bands, with further indications that the center of the mid-infrared structure is displaced with respect to the optical center of the NSC. We discuss five potential scenarios, none of them fully explaining both the observed photometry and structure. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.13318.md
    + _build/html/tmp_2211.13318/./binaryProperties.png
    + _build/html/tmp_2211.13318/./etaCumMass_500.png
    + _build/html/tmp_2211.13318/./binaryAU.png
exported in  _build/html/2211.13426.md
    + _build/html/tmp_2211.13426/./CMDs_NGC7496_v4_r.png
    + _build/html/tmp_2211.13426/./galaxy_plus_region.png
    + _build/html/tmp_2211.13426/./ngc7496_rgb_co_h_alpha_v2.png
exported in  _build/html/2211.13811.md
    + _build/html/tmp_2211.13811/./figs/G35.6-0.4.png
    + _build/html/tmp_2211.13811/./figs/G29.6+0.1.png
    + _build/html/tmp_2211.13811/./figs/G31.5-0.6.png
    + _build/html/tmp_2211.13811/./figs/G32.4+0.1.png
    + _build/html/tmp_2211.13811/./figs/G32.8-0.1.png
    + _build/html/tmp_2211.13811/./figs/spidx_dist2.png
exported in  _build/html/2211.14132.md
    + _build/html/tmp_2211.14132/./figs/lithium_top6li_all11subgs_lifeh_g19.png
    + _build/html/tmp_2211.14132/./figs/lithium_top6li_all11subgs_liteff.png
    + _build/html/tmp_2

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\hubert}[1]{{\color{magenta}#1}}$
$\newcommand{\brooke}[1]{{\color{teal}#1}}$
$\newcommand{\response}[1]$
$\newcommand{\responsetwo}[1]$
$\newcommand{\baselinestretch}{1.1}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\hubert$}[1]{{\color{magenta}#1}}$
$\newcommand{$\brooke$}[1]{{\color{teal}#1}}$
$\newcommand{$\response$}[1]$
$\newcommand{$\response$two}[1]$
$\newcommand{$\baselinestretch$}{1.1}$</div>



<div id="title">

# High Resolution Study of Planetesimal Formation by Gravitational Collapse of Pebble Clouds

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.13318-b31b1b.svg)](https://arxiv.org/abs/2211.13318)<mark>Appeared on: 2022-11-28</mark> - _Accepted for publication in ApJ_

</div>
<div id="authors">

Brooke Polak, <mark>Hubert Klahr</mark>

</div>
<div id="abstract">

**Abstract:** Planetary embryos are built through the collisional growth of 10-100 km sized objects called planetesimals, a formerly large population of objects, of which asteroids, comets and Kuiper-Belt objects represent the leftovers from planet formation in our solar system.Here, we follow the paradigm that turbulence created over-dense pebble clouds, which then collapse under their own self-gravity.We use the multi-physics code GIZMO  to model the pebble cloud density as a continuum, with a polytropic equation of state to account for collisional interactions and capturing the phase transition to a quasi-incompressible "solid" object, i.e. a planetesimal in hydrostatic equilibrium.Thus we study cloud collapse effectively at the resolution of the forming planetesimals, allowing us to derive an initial mass function for planetesimals in relation to the total pebble mass of the collapsing cloud.The redistribution of angular momentum in the collapsing pebble cloud is the main mechanism leading to multiple fragmentation. The angular momentum of the pebble cloud and thus the centrifugal radius increases with distance to the sun, but the solid size of the forming planetesimals is constant. Therefore we find that with increasing distance to the sun, the number of forming planetesimals per pebble cloud increases. For all distances the formation of binaries occurs within higher hierarchical systems.The size distribution is top heavy and can be described with a Gaussian distribution of planetesimal mass. For the asteroid belt, we can infer a most likely size of 125 km, all stemming from pebble clouds of equivalent size 152 km.

</div>

<div id="div_fig1">

<img src="tmp_2211.13318/./binaryProperties.png" alt="Fig22" width="100%"/>

**Figure 22. -** Properties of the binaries formed in our simulations. Circles, triangles, and squares are the simulated binaries and plus signs are observations of KBOs. The color represents the semi-major axis of the objects around a 1M$_\odot$ central star. The opaque circles are the binaries that have completed at least one orbital radius, and the transparent squares are the binaries that have been detected but haven't had time to complete an orbit. The transparent triangles are the moons, with the lines indicating to which binary they belong. None of the moons have completed an orbit. Upper left plot shows the scaled binary semi-major axis against the binary size ratio. Upper right shows the angular momentum scaled to the Hill angular momentum $L_H=$. Lower left shows the eccentricities. The lines denote the pericenter distances $\beta=a_b(1-e_b)/(R_1+R_2)=$35 (dotted) and 2 (dashed) which represent the binary separation resolving limit of the \citet{Nesvorny2021} simulations and our current model, respectively. This means our current model can resolve much tighter binaries than previous models. Lower right shows the binary orbit inclination angle from the initial rotation axis of the cloud. (*fig:binaryProperties*)

</div>
<div id="div_fig2">

<img src="tmp_2211.13318/./etaCumMass_500.png" alt="Fig3" width="100%"/>

**Figure 3. -** Cumulative mass distribution of clumps in units of the initial cloud mass. Color represents orbital radius. (*fig:etaCumMass*)

</div>
<div id="div_fig3">

<img src="tmp_2211.13318/./binaryAU.png" alt="Fig11" width="100%"/>

**Figure 11. -** Binary proximity factor $\zeta$ against heliocentric semi-major axis. Circles, triangles, and squares are the simulated binaries and plus signs are observations of KBOs. The color represents the semi-major axis of the objects around a 1M$_\odot$ central star. The triangles are moons, with lines indicating to which binary they belong. The opaque circles have completed at lest one orbit, and the transparent squares (and triangles) have not. For a contact binary, $\zeta=1$. The maximum proximity factor $\zeta_0$ for a given semi-major axis is shown by the dashed black line. This factor corresponds to the binary that would form if all the angular momentum from the initial cloud went into a single equal mass binary. (*fig:binaryAU*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\sfr}{M_{\odot} yr^{-1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\msun$}{M_{\odot}}$
$\newcommand{$\sfr$}{M_{\odot} yr^{-1}}$</div>



<div id="title">

# m PAH emission

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.13426-b31b1b.svg)](https://arxiv.org/abs/2211.13426)<mark>Appeared on: 2022-11-28</mark> - _12 pages, 6 figures_

</div>
<div id="authors">

M. Jimena Rodríguez, et al. -- incl., <mark>Janice C. Lee</mark>, <mark>Eva Schinnerer</mark>, <mark>Daizhong Liu</mark>, <mark>Kathryn~Kreckel</mark>, <mark>Annie~Hughes</mark>

</div>
<div id="abstract">

**Abstract:** The earliest stages of star formation occur enshrouded in dustand are not observable in the optical.Here we leverage the extraordinary new high-resolution infrared imaging from JWST to begin the study of dust-embedded star clusters in nearby galaxies throughout the local volume.  We present atechnique for identifying dust-embedded clusters in NGC 7496 (18.7 Mpc), the first galaxy to be observed by the PHANGS-JWST Cycle 1 Treasury Survey.We select sources that have strong 3.3$\mu$m PAH emission based on a$\rm F300M-F335M$color excess, and identify 67 candidate embedded clusters.  Only eight of these are found in the PHANGS-HST optically-selected cluster catalog and all are young (six have SED-fit ages of$\sim1$Myr).  We find that this sample of embedded cluster candidates may significantly increase the census of young clusters in NGC 7496 from the PHANGS-HST catalog -- the number of clusters younger than$\sim$2 Myr could be increased by a factor of two.  Candidates are preferentially located in dust lanes, and are coincident with peaks in PHANGS-ALMA CO (2-1)maps. We take a first look at concentration indices, luminosity functions, SEDs spanning from 2700\AAto 21$\mu$m, and stellar masses (estimated to be between$\sim10^4-10^5 M_{\odot}$).  The methods tested here provide a basis for future work to derive accurate constraints on the physical properties of embedded clusters, characterize the completeness of cluster samples, and expand analysis to all 19 galaxies in the PHANGS-JWST sample, which will enable basic unsolved problems in star formation and cluster evolution to be addressed.

</div>

<div id="div_fig1">

<img src="tmp_2211.13426/./CMDs_NGC7496_v4_r.png" alt="Fig4" width="100%"/>

**Figure 4. -** Color-magnitude diagram $\rm F335M$ vs. $\rm F300M-F335M$ for NGC 7496. Magnitudes are in the AB system. In all panels the black dots are $\rm F335M$ sources detected with Photutils {\it find\_peaks}. Density contours for the densest parts of this diagram are shown in panels {\it a} and {\it b}. In panel {\it a)} yellow stars show the 12 prototype embedded clusters selected in our initial visual inspection. Based on the location in the CMD of these candidates, we select 67 embedded candidates (cyan circles). The dashed lines show the selection criteria: $\rm F300M-F335M >$0.3 and F335M$<$24. The magenta box indicates a region where scatter due to the uncertainties in the color is large, and is chosen for the purposes of illustrating the spatial distribution in Fig. \ref{Fig:color_CO_alpha}.
Panel {\it b)} shows the same extended sample of 67 candidates but differentiated by category: embedded objects (red), intermediate or partially embedded (green), and visible or exposed (blue).
Panels {\it c} and {\it d} show for comparison the location in the CMD sources from the PHANGS-HST optical cluster catalog.
Panel {\it c)} show clusters older than 100 Myr (magenta). As expected, these clusters fall around $\rm F300M-F335M=0$, indicating that these old sources do not have PAH emission. This panel also shows clusters with SED-fit age estimates between 10-100 Myr (green crosses) and 5-10 Myr (yellow diamonds), which also appear around $\rm F300M-F335M=0$. In contrast, about half of the clusters younger than 2 Myr shown in panel {\it d)}(small cyan crosses) are PAH emitters. Panel {\it d)} also shows clusters between 2 and 5 Myr (orange squares), which mostly have little measured PAH emission. (*fig:cmd*)

</div>
<div id="div_fig2">

<img src="tmp_2211.13426/./galaxy_plus_region.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
The upper panel shows the PHANGS-HST WFC3 $\rm F555M$ image of NGC 7496.
The sample of 67 PHANGS-JWST 3.3 $\mu$m selected star cluster candidates are plotted with different colors according to their categories: embedded (red), partially embedded (green) and visible (blue).
The locations of optically-selected clusters from the PHANGS-HST catalog are also shown, and distinguished by SED-fit age (black squares for $>10$ Myr and black circles for $<10$ Myr). Only clusters brighter than $\rm F300M < 25$ are included.
The bottom panels show images in the HST-WFC3, JWST-NIRCam, and JWST-MIRI filters, for the $10" \times 10"$ region indicated by the white box in the upper panel. The central wavelength of each filter is indicated in the panel label.
The colored circles are the same as in the upper panel. Clusters marked by an additional yellow circle are part of our prototype sample (see text for details).
The comparison of this region in the optical HST images and IR from JWST, clearly shows that the candidates fall along a dust lane, largely obscured in the HST bands.
\label{fig:galaxy+reg} (*fig:galaxy+reg*)

</div>
<div id="div_fig3">

<img src="tmp_2211.13426/./ngc7496_rgb_co_h_alpha_v2.png" alt="Fig6" width="100%"/>

**Figure 6. -** {\it Left:} Combined HST F555W (blue) and JWST F335M (red) image. The green dots are sources detected in F335M that satisfy our color cut $\rm F300M-F335M>0.3$(Sect. \ref{subsect:selectioncriteria}), but before performing any cut in magnitude.  Sources shown in green without a magenta circle are the strongest PAH emitters, and are mostly found in the main dust lanes within the spiral arms.
Sources with an additional magenta circle are those that have $\rm F300M-F335M$ between 0.3 and 0.7, and $\rm F335M>24.5$, i.e., they are both fainter and have larger color uncertainties (magenta box in Fig. \ref{fig:cmd}). This figure shows that when we select only the brightest sources with the largest color excesses, sources which tend to coincide with minor dust lanes in the galaxy, i.e., potentially fainter embedded clusters, are excluded.
{\it Middle}: CO(2-1) ALMA map, with resolution of $\sim 1"$ with the 67 embedded cluster candidates (red circles).
{\it Right}: Muse H$_{\alpha}$ map with resolution of $\sim 0$\farcs$$7 and the 67 candidates. Note that the sample is well-correlated with $H_{\alpha}$ and CO(2-1) peaks.  (*Fig:color_CO_alpha*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\dok}{\citetalias{2021A&A...651A..86D}}$
$\newcommand{\loren}{\citetalias{2017A&A...605A..58A}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\dok$}{\citetalias{2021A&A...651A..86D}}$
$\newcommand{$\loren$}{\citetalias{2017A&A...605A..58A}}$</div>



<div id="title">

# A global view on star formation: The GLOSTAR Galactic plane survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.13811-b31b1b.svg)](https://arxiv.org/abs/2211.13811)<mark>Appeared on: 2022-11-28</mark> - _To be published in A&A. 21 pages, 15 figures_

</div>
<div id="authors">

R. Dokara, et al. -- incl., <mark><mark>H. Beuther</mark></mark>, <mark><mark>P. Müller</mark></mark>

</div>
<div id="abstract">

**Abstract:** While over 1000 supernova remnants (SNRs) are estimated to exist in the Milky Way, only less than 400 have been found to date.  In the context of this apparent deficiency, more than 150 SNR candidates were recently identified in the D-configuration Very Large Array (VLA-D) continuum images of the 4--8 GHz global view on star formation (GLOSTAR) survey, in the Galactic longitude range$-2\degree<l<60\degree$.We attempt to find evidence of nonthermal synchrotron emission from 35 SNR candidates in the region of Galactic longitude range$28\degree<l<36\degree$, and also to study the radio continuum emission from the previously confirmed SNRs in this region.Using the short-spacing corrected GLOSTAR VLA-D+Effelsberg images, we measure the${\sim}6$GHz total and linearly polarized flux densities of the SNR candidates and the SNRs that were previously confirmed.  We also attempt to determine the spectral indices by measuring flux densities from complementary Galactic plane surveys and from the temperature-temperature plots of the GLOSTAR-Effelsberg images.We provide evidence of nonthermal emission from four candidates that have spectral indices and polarization consistent with a SNR origin, and, considering their morphology, we are confident that three of these (G28.36+0.21, G28.78-0.44, and G29.38+0.10) are indeed SNRs.  However, about$25\%$of the candidates (8 out of 35) have spectral index measurements that indicate thermal emission, and the rest of them are too faint to have a good constraint on the spectral index yet.Additional observations at longer wavelengths and higher sensitivities will shed more light on the nature of these candidates.  A simple Monte-Carlo simulation reiterates the view that future studies must persist with the current strategy of searching for SNRs with small angular size to solve the problem of the Milky Way's missing SNRs.

</div>

<div id="div_fig1">

<img src="tmp_2211.13811/./figs/G35.6-0.4.png" alt="Fig5" width="100%"/>

**Figure 5. -** SNR G35.6-0.4: the top left and right panels show the GLOSTAR combination images of total and linearly polarized intensity.  The red dotted circles represent the two shell-like structures identified at 610 MHz by \citet{2014A&A...561A..56P}, while the black diamond marks the position of the recombination line discovered by \citet{1989ApJS...71..469L}.  The TT-plot from GLOSTAR-Effelsberg images and the flux density spectrum are presented in the bottom left and right panels respectively.  (*fig:G35.6-0.4*)

</div>
<div id="div_fig2">

<img src="tmp_2211.13811/./figs/G29.6+0.1.png" alt="Fig3.1" width="25%"/><img src="tmp_2211.13811/./figs/G31.5-0.6.png" alt="Fig3.2" width="25%"/><img src="tmp_2211.13811/./figs/G32.4+0.1.png" alt="Fig3.3" width="25%"/><img src="tmp_2211.13811/./figs/G32.8-0.1.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** From top to bottom: G29.6+0.1, G31.5-0.6, G32.4+0.1, and G32.8-0.1.  Left panels show the GLOSTAR combination images.  The TT-plot from GLOSTAR-Effelsberg images, and the flux density spectrum using the GLOSTAR combination images and ancillary data are presented in the middle and right panels respectively.   (*fig:ksnrs*)

</div>
<div id="div_fig3">

<img src="tmp_2211.13811/./figs/spidx_dist2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Flux density spectral indices ($\alpha_{\mathrm{FD}}$) of the candidate SNRs being studied in this work.  Candidates with lower limits are represented by upward arrows.  Since G29.38+0.10 has a spectral break, both the spectral indices are shown.  (*fig:spidxdist*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\feh}{\langle \rm{[Fe/H]} \rangle}$
$\newcommand{\ali}{A(Li)}$
$\newcommand{\medt}{\overline{t}}$
$\newcommand{\sfehtot}{\sigma(\rm{[Fe/H]})_{\rm tot}}$
$\newcommand{\sfehtop}{\sigma(\rm{[Fe/H]})_{\rm top6}}$
$\newcommand{\smgfetot}{\sigma(\rm{[Mg/Fe]})_{\rm tot}}$
$\newcommand{\smgfetop}{\sigma(\rm{[Mg/Fe]})_{\rm top6}}$
$\newcommand$
$\newcommand$
$\newcommand{\mlld}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\guillaume}[1]{\textcolor{blue}{#1}}$
$\newcommand{\laura}[1]{\textcolor{red}{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\feh$}{\langle \rm{[Fe/H]} \rangle}$
$\newcommand{$\ali$}{A(Li)}$
$\newcommand{$\medt$}{\overline{t}}$
$\newcommand{$\sfehtot$}{\sigma(\rm{[Fe/H]})_{\rm tot}}$
$\newcommand{$\sfehtop$}{\sigma(\rm{[Fe/H]})_{\rm top6}}$
$\newcommand{$\smgfetot$}{\sigma(\rm{[Mg/Fe]})_{\rm tot}}$
$\newcommand{$\smgfetop$}{\sigma(\rm{[Mg/Fe]})_{\rm top6}}$
$\newcommand$
$\newcommand$
$\newcommand{$\mlld$}[1]{\textcolor{magenta}{#1}}$
$\newcommand{$\guillaume$}[1]{\textcolor{blue}{#1}}$
$\newcommand{$\laura$}[1]{\textcolor{red}{#1}}$</div>



<div id="title">

# -ESO Survey: probing the lithium abundances in old  metal-rich dwarf stars in the Solar vicinity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.14132-b31b1b.svg)](https://arxiv.org/abs/2211.14132)<mark>Appeared on: 2022-11-28</mark> - _8 pages, 3 figures. Accepted for publication in A&A Letters. Abridged abstract to fit ArXiv's requirements_

</div>
<div id="authors">

M. L. L. Dantas, et al. -- incl., <mark><mark>G. Guiglion</mark></mark>, <mark><mark>U. Heiter</mark></mark>

</div>
<div id="abstract">

**Abstract:** Lithium (Li) is a fragile element that is produced in a variety of sites, but can also be very easily depleted in stellar photospheres. Radial migration has been reported to explain the decrease of the upper envelope of Li measurements observed for relatively old metal-rich dwarf stars in some surveys.We test a scenario in which radial migration could affect the Li abundance pattern of dwarf stars in the solar neighbourhood. This may confirm that the Li abundances in these stars can not serve as a probe for the Li abundance in the interstellar medium. In other words, to probe the evolution of the Li abundance in the local interstellar medium (ISM), it is crucial that stellar intruders are identified and removed from the adopted sample.We use the high-quality data (including Li abundances) from the 6\textsuperscript{th}internal Data Release of the\emph{Gaia}-ESO survey. In this sample, we group stars by similarity in chemical abundances via hierarchical clustering. Our analysis treats both measured Li abundances and upper limits.The Li envelope of the previously identified radially migrated stars is well below the benchmark meteoritic value (<3.26 dex); the star with the highest detected abundance has A(Li) = 2.76 dex. This confirms the previous trends observed for old dwarf stars (median ages$\sim$8 Gyr), where Li decreases for[Fe/H]$\gtrsim$0.This result acts as supporting evidence that the abundance of Li measured in the upper envelope of old dwarf stars should not be considered a proxy for the interstellar medium Li. Our scenario also indicates that the stellar yields for[M/H]>0 should not be decreased, as recently proposed in the literature. Our study backs the recent studies that claimed that old dwarfs on the hot side of the dip are efficient probes of the ISM abundance of Li, provided atomic diffusion does not lower significantly the initial Li abundance in the atmospheres of metal-rich objects.

</div>

<div id="div_fig1">

<img src="tmp_2211.14132/./figs/lithium_top6li_all11subgs_lifeh_g19.png" alt="Fig1.1" width="50%"/><img src="tmp_2211.14132/./figs/lithium_top6li_all11subgs_liteff.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** \emph{Left panel:}$\langle{\rm A(Li)} \rangle$ vs $\feh$ for the super-solar groups of the sample, split into those with direct detection of Li (LiD -- round markers) and those with upper limit estimate (LiUL -- inverted triangle markers). In this representation, only the median of top 6 stars with the highest $\ali$ are shown in each marker. Dark magenta markers have been added to represent the meteoritic ({\tiny{$\bigotimes$}}) and Solar photospheric ({\tiny $\bigodot$}) values. The median ages of the stars represented by each marker are displayed on the legend. The black star-shaped markers display the data from \citet{guiglion_2016}. It is worth mentioning that the errors associated with $\ali$ are very small and that is why they are not seen. \emph{Right panel:}$\langle{\rm A(Li)} \rangle$ vs $\langle T_{\rm eff} \rangle$: it is possible to see that $\ali$ seems to decrease with decreasing $T_{\rm eff}$. Indeed, the warmer temperatures seem to have a protective effect on $\ali$ due to their thinner convective layers. A straight black dotted line is added to depict an approximate estimation of $T_{\rm{eff}}$(i.e. at $\sim$5700K) for unmodified $\ali$ as shown in \citet[][]{romano2021}. (*fig:li_feh_top6*)

</div>
<div id="div_fig2">

<img src="tmp_2211.14132/./figs/MMR_solar_feh_li.png" alt="Fig2" width="100%"/>

**Figure 2. -** $\ali$ vs [Fe/H] in the shape of a scatter-plot and a 2D-Gaussian kernel density plot the MMR subgroups (from 6 to 10) in order of increasing $\feh$. Round and inverted-triangle markers depict, respectively, LiD and LiUL measurements. The 2D-kernel densities are either in the same colour as the scatter markers (for LiD) or in grey (for LiUL). 2D-kernel densities are not shown for subgroups 9 and 10 in the case of LiD due to the low number of stars with detected Li in each of these subgroups (2 and 1, respectively). Similarly to Fig. \ref{fig:li_feh_top6}, additional markers were added to depict both the meteoritic and Solar photospheric $\ali$. The MMR group colours are the same as in \citet{Dantas2022}. The errors are omitted to ease the visualisation. The median ages of each MMR subgroup are depicted at the top of each subplot; it is worth noting that subgroup 10 (purple) has a smaller $\overline{t}$ than depicted in \citet[][]{Dantas2022}. The reason for this difference is due to the stars removed with missing Li. (*fig:mmr_feh_li*)

</div>
<div id="div_fig3">

<img src="tmp_2211.14132/./figs/mmr_li_teff.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\ali$ vs $T_{\rm{eff}}$ for all MMR stars. Stars with LiD are represented by the round markers whereas those with LiUL are displayed using inverted triangles. The colours that differentiate each MMR subgroup are the same as in Fig. \ref{fig:mmr_feh_li}. A straight black dotted line is added to depict an approximate estimation of $T_{\rm{eff}}$(i.e. at $\sim$5700K) for unmodified $\ali$ as shown in \citet[][]{romano2021}. (*fig:mmr_li_teff*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

102  publications in the last 7 days.
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/html/2211.11767.md
	 _build/html/2211.09840.md
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers